In [1]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql

In [3]:
#reading dataset
cov=pd.read_csv('Coverage_data.csv')
sch=pd.read_csv('Schedule.csv')
intro=pd.read_csv('Intro.csv')
rep=pd.read_csv('report.csv')
inc=pd.read_csv('incidence_rate.csv')

In [23]:
# make csv file of them 
cov.to_csv('Coverage_data.csv')
sch.to_csv('Schedule.csv')
intro.to_csv('Intro.csv')
rep.to_csv('report.csv')
inc.to_csv('incidence_rate.csv')

In [3]:
cov.head()

,GROUP,CODE,NAME,YEAR,ANTIGEN,ANTIGEN_DESCRIPTION,COVERAGE_CATEGORY,COVERAGE_CATEGORY_DESCRIPTION,TARGET_NUMBER,DOSES,COVERAGE
0,COUNTRIES,ABW,Aruba,2023.0,BCG,BCG,ADMIN,Administrative coverage,1044.0,945.0,90.52
1,COUNTRIES,ABW,Aruba,2023.0,BCG,BCG,OFFICIAL,Official coverage,1044.0,945.0,90.52
2,COUNTRIES,ABW,Aruba,2023.0,DIPHCV4,"Diphtheria-containing vaccine, 4th dose (1st b...",ADMIN,Administrative coverage,1044.0,945.0,90.52
3,COUNTRIES,ABW,Aruba,2023.0,DIPHCV4,"Diphtheria-containing vaccine, 4th dose (1st b...",OFFICIAL,Official coverage,1219.0,1008.0,90.52
4,COUNTRIES,ABW,Aruba,2023.0,DIPHCV5,"Diphtheria-containing vaccine, 5th dose (2nd b...",ADMIN,Administrative coverage,1219.0,1008.0,82.69


In [4]:
cov.shape #size

(399859, 11)

In [5]:
cov.isnull().sum() # getting the null values

GROUP                            0
CODE                             0
NAME                             0
YEAR                             1
ANTIGEN                          0
ANTIGEN_DESCRIPTION              0
COVERAGE_CATEGORY                0
COVERAGE_CATEGORY_DESCRIPTION    0
TARGET_NUMBER                    1
DOSES                            1
COVERAGE                         1
dtype: int64

In [6]:
cov=cov[~cov['YEAR'].isnull()]

In [7]:
cov.isnull().sum()

GROUP                            0
CODE                             0
NAME                             0
YEAR                             0
ANTIGEN                          0
ANTIGEN_DESCRIPTION              0
COVERAGE_CATEGORY                0
COVERAGE_CATEGORY_DESCRIPTION    0
TARGET_NUMBER                    0
DOSES                            0
COVERAGE                         0
dtype: int64

In [67]:
sch.isnull().sum()

Unnamed: 0               0
ISO_3_CODE               0
COUNTRYNAME              0
WHO_REGION               0
YEAR                     0
VACCINECODE              0
VACCINE_DESCRIPTION      0
SCHEDULEROUNDS           0
TARGETPOP                0
TARGETPOP_DESCRIPTION    0
GEOAREA                  0
AGEADMINISTERED          0
SOURCECOMMENT            0
dtype: int64

In [61]:
sch=sch[~sch['SOURCECOMMENT'].isnull()]

In [65]:
sch=sch[~sch['YEAR'].isnull()]

In [9]:
rep.isnull().sum()

Unnamed: 0             0
GROUP                  0
CODE                   0
NAME                   0
YEAR                   0
DISEASE                0
DISEASE_DESCRIPTION    0
CASES                  0
dtype: int64

In [7]:
rep=rep[~rep['CODE'].isnull()]

In [11]:
# creating sql connection
conn=pymysql.connect(host='localhost',
                     user='root',
                     password='789456ujjwal@',
                     database='vac_analysis'
                    )
cursor=conn.cursor()
print('connected')

connected


In [13]:
cursor.execute("Drop table REPORT")

0

In [10]:
# creating table coverage
cursor.execute("""
    CREATE TABLE IF NOT EXISTS COVERAGE(
        GROUP_NAME varchar(100),
        CODE varchar(100),
        NAME varchar(255),
        YEAR Decimal(6,1),
        ANTIGEN  Text,
        ANTIGEN_DESCRIPTION Text,
        COVERAGE_CATEGORY varchar(100),
        COVERAGE_CATEGORY_DESCRIPTION Text,
        TARGET_NUMBER Decimal(20,2),
        DOSES   Decimal(20,2),
        COVERAGE Decimal(10,2)
        )
        """)


0

In [53]:
# creating table incidence

cursor.execute("""
CREATE TABLE IF NOT EXISTS INCIDENCE(
            GROUP_Name VARCHAR(100),
            CODE VARCHAR(100),
            NAME VARCHAR(255),
            YEAR Decimal(6,1),
            DISEASE VARCHAR(100),
            DISEASE_DESCRIPTION VARCHAR(150),
            DENOMINATOR VARCHAR(100),
            INCIDENCE_RATE DECIMAL(10,5)
        )
    """
              )


0

In [77]:
# creating table schedule
cursor.execute( """
        CREATE TABLE IF NOT EXISTS SCHEDULE (
            ISO_3_CODE VARCHAR(100),
            COUNTRYNAME VARCHAR(100),
            WHO_REGION VARCHAR(100),
            YEAR Decimal(6,1),
            VACCINECODE VARCHAR(50),
            VACCINE_DESCRIPTION TEXT,
            SCHEDULEROUNDS TEXT,
            TARGETPOP VARCHAR(50),
            TARGETPOP_DESCRIPTION TEXT,
            GEOAREA VARCHAR(50),
            AGEADMINISTERED VARCHAR(255),
            SOURCECOMMENT TEXT
        )
        """
              )

0

In [7]:
# creating table intro
cursor.execute("""
            CREATE TABLE IF NOT EXISTS INTRO (
            ISO_3_CODE VARCHAR(5),
            COUNTRYNAME TEXT,
            WHO_REGION VARCHAR(10),
            YEAR DECIMAL(10,1),
            DESCRIPTION TEXT
        )
    """)

0

In [15]:
# creating table report
cursor.execute("""
        CREATE TABLE IF NOT EXISTS REPORT (
            GROUP_NAME VARCHAR(100),
            CODE VARCHAR(10),
            NAME VARCHAR(255),
            YEAR DECIMAL(6,1),
            DISEASE TEXT,
            DISEASE_DESCRIPTION TEXT,
            CASES DECIMAL(10,1)
        )
    """)

0

In [ ]:
#inserting values to it 

In [ ]:
insert_query="""
    Insert into COVERAGE (GROUP_NAME,CODE,NAME,YEAR,ANTIGEN,ANTIGEN_DESCRIPTION,COVERAGE_CATEGORY,COVERAGE_CATEGORY_DESCRIPTION,TARGET_NUMBER,DOSES,COVERAGE)
    Values(%s ,%s ,%s ,%s ,%s ,%s ,%s ,%s ,%s ,%s ,%s)
    """
for i,row in cov.iterrows():
    Values=(row['GROUP'],row['CODE'],row['NAME'],row['YEAR'],row['ANTIGEN'],row['ANTIGEN_DESCRIPTION'],row['COVERAGE_CATEGORY'],row['COVERAGE_CATEGORY_DESCRIPTION']
           ,row['TARGET_NUMBER'],row['DOSES'],row['COVERAGE'])
    cursor.execute(insert_query,Values) 
    conn.commit()

In [ ]:
insert_query="""
        INSERT INTO INCIDENCE (GROUP_Name, CODE, NAME, YEAR, DISEASE, DISEASE_DESCRIPTION, DENOMINATOR, INCIDENCE_RATE)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """
for i,row in inc.iterrows():
    values=(row['GROUP'],row['CODE'],row['NAME'],row['YEAR'],row['DISEASE'],row['DISEASE_DESCRIPTION'],row['DENOMINATOR'],row['INCIDENCE_RATE'])
    cursor.execute(insert_query,values)
    conn.commit()
    

In [79]:
insert_query="""
        INSERT INTO SCHEDULE (ISO_3_CODE, COUNTRYNAME, WHO_REGION, YEAR, VACCINECODE, VACCINE_DESCRIPTION, 
        SCHEDULEROUNDS, TARGETPOP, TARGETPOP_DESCRIPTION, GEOAREA, AGEADMINISTERED, SOURCECOMMENT)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
for i,row in sch.iterrows():
    values=(row['ISO_3_CODE'],row['COUNTRYNAME'],row['WHO_REGION'],row['YEAR'],row['VACCINECODE'],row['VACCINE_DESCRIPTION'],
            row['SCHEDULEROUNDS'],row['TARGETPOP'],row['TARGETPOP_DESCRIPTION'],row['GEOAREA'],row['AGEADMINISTERED'],row['SOURCECOMMENT'])
    cursor.execute(insert_query,values)
    conn.commit()

In [ ]:
insert_query="""
        INSERT INTO INTRO (ISO_3_CODE, COUNTRYNAME, WHO_REGION, YEAR, DESCRIPTION)
        VALUES (%s, %s, %s, %s, %s)
    """
for i,row in intro.iterrows():
    values=(row['ISO_3_CODE'],row['COUNTRYNAME'],row['WHO_REGION'],row['YEAR'],row['DESCRIPTION'])
    cursor.execute(insert_query,values)
    conn.commit()


In [17]:
insert_query="""
        INSERT INTO REPORT (GROUP_NAME, CODE, NAME, YEAR, DISEASE, DISEASE_DESCRIPTION, CASES)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
for i,row in rep.iterrows():
    values=(row['GROUP'],row['CODE'],row['NAME'],row['YEAR'],row['DISEASE'],row['DISEASE_DESCRIPTION'],row['CASES'])
    cursor.execute(insert_query,values)
    conn.commit()